In [1]:
import os
import openai

from dotenv import load_dotenv
from ToolGPT import ChatGPTWithFunctions

from functions.crypto import get_crypto_peak_price
from functions.my_machine import my_machine_os

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Prompting Open AI's GPT `3.5-turbo` Model

In our prompt we want to get some resent in


In [4]:
prompt = "What was the peak price of Bitcoin yesterday?"
chat_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}])
print(chat_completion.choices[0].message.content)

I apologize, as an AI language model, I do not have real-time access to current market data. Kindly check reliable cryptocurrency exchanges or financial news websites for the latest updates on the peak price of Bitcoin yesterday.


In [5]:
prompt = "What was the peak price of Bitcoin yesterday?"
wrapper = ChatGPTWithFunctions()
ans = wrapper.prompt_with_functions(prompt, [get_crypto_peak_price])
print(ans)

What was the peak price of Bitcoin yesterday?
Received FUNCTION_CALL signal
{
  "cryptocurrency": "Bitcoin"
}
The peak price of Bitcoin yesterday was $26,391.23.
Received STOP signal
None


In [3]:
prompt = "How do I install docker onto my machine?"
chat_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}])
print(chat_completion.choices[0].message.content)

As an AI language model, I don't have access to your specific machine. However, I can provide you with the general steps to install Docker on your machine:

1. Go to the Docker website and download the appropriate installer for your operating system. 
2. Follow the installation instructions for your specific OS.
3. Once installed, open a command prompt or terminal window and type "docker version" to ensure that Docker was installed correctly and that the daemon is running.

Congratulations, you now have Docker installed on your machine!


In [2]:
prompt = "How do I install docker onto my machine?"
wrapper = ChatGPTWithFunctions()
ans = wrapper.prompt_with_functions(prompt, [my_machine_os])
print(ans)

How do I install docker onto my machine?
Received FUNCTION_CALL signal
{
  "my_machine": "my machine"
}
To install Docker on Linux, you can follow these steps:

1. Update the package index on your machine:
   ```
   sudo apt update
   ```

2. Install the necessary packages to allow apt to use a repository over HTTPS:
   ```
   sudo apt install apt-transport-https ca-certificates curl software-properties-common
   ```

3. Add the official Docker GPG key:
   ```
   curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
   ```

4. Add the Docker repository to your system's sources.list:
   ```
   echo "deb [arch=amd64 signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
   ```

5. Update the package index again:
   ```
   sudo apt update
   ```

6. Install Docker:
   ```
   sudo apt insta